# Initiailize VW executor

In [ ]:
from VwPipeline import Loggers, Handlers
from VwPipeline.VwCache import VwCache
from VwPipeline.Vw import Vw
from VwPipeline.VwOpts import dimension, product
import pandas as pd

#your vw path
vw_path = r'C:\vw\vw.exe'

cache = VwCache(r'_cache')
vw = Vw(vw_path,
    cache,
    handlers=[Handlers.WidgetHandler()],
#    loggers=[Loggers.ConsoleLogger('DEBUG')]
    )

# Get the data

In [ ]:
inputs = [r'C:\data\cb\1.json',
    r'C:\data\cb\2.json',
#    r'C:\data\cb\3.json',
    ]
inputs

# Define options grid and train

In [ ]:
opts = product(
    dimension('#base', ['--ccb_explore_adf -P 10000 --preserve_performance_counters --save_resume --dsjson']),
    dimension('--cb_type', ['ips', 'mtr']),
)
opts

In [ ]:
result = vw.train(inputs, opts)
result

## Get access to losses and job artifacts

In [ ]:
print(f'Loss[0]: {result[0].Loss}')
print(f'Populated options: {result[0].Populated}')

# Pandas wrappers

In [ ]:
opts = pd.DataFrame(product(
    dimension('#base', ['--ccb_explore_adf -P 10000 --preserve_performance_counters --save_resume']),
    dimension('--cb_type', ['ips', 'mtr']),
    dimension('#format', ['', '--dsjson'])
))
opts

In [ ]:
result = vw.train(inputs, opts)
result

## Investigate failure

In [ ]:
result[~result['!Loss'].notnull()].iloc[0]['!Job'].Tasks[0].stdout()

## Explore metrics 

In [ ]:
result.iloc[1]['!Metrics'].head()

In [ ]:
import matplotlib.pyplot as plt
[(-r['!Metrics']['loss_per_example']).plot(label=f"{r['!Job'].Name}", figsize=(10,6)) for i, r in result[result['!Loss'].notnull()].iterrows()]
plt.legend()